In [1]:
!pip install numpy==1.23.1
!pip install torchvision==0.14.1

  Using cached torchvision-0.14.1-cp38-cp38-manylinux2014_aarch64.whl (12.9 MB)
ERROR: pytorch2timeloop 0.2 has requirement numpy==1.21, but you'll have numpy 1.23.1 which is incompatible.
ERROR: pytorch2timeloop 0.2 has requirement torchvision==0.14, but you'll have torchvision 0.14.1 which is incompatible.
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.14.0
    Uninstalling torchvision-0.14.0:
      Successfully uninstalled torchvision-0.14.0


In [1]:
# some basic imports and setups
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.models as models
import matplotlib.pyplot as plt
%matplotlib inline
import urllib
from PIL import Image
from torchvision import transforms

/home/workspace/.local/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


In [2]:
#load model - AlexNet
from torch.utils.model_zoo import load_url as load_state_dict_from_url
model_url = 'https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth'

class AlexNet_custom(nn.Module):

    def __init__(self, num_classes=1000):
        super(AlexNet_custom, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6)) # in case input image is larger
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x

# load pretrained weights
def alexnet_custom(path = "model_weights/pretrained_AlexNet.pt", pretrained=True, progress=True, **kwargs):
    model = AlexNet_custom(**kwargs)
#     net = torch.jit.load(path)
    net = torch.load(path)
    params = net.state_dict()
#     state_dict = load_state_dict_from_url(model_url, progress=progress)
#     model.load_state_dict(state_dict)
    model.load_state_dict(params)
    return model

from collections import namedtuple, OrderedDict, defaultdict
class VGG(nn.Module):
  ARCH = [64, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']

  def __init__(self) -> None:
    super().__init__()

    layers = []
    counts = defaultdict(int)

    def add(name: str, layer: nn.Module) -> None:
      layers.append((f"{name}{counts[name]}", layer))
      counts[name] += 1

    in_channels = 3
    for x in self.ARCH:
      if x != 'M':
        # conv-bn-relu
        add("conv", nn.Conv2d(in_channels, x, 3, padding=1, bias=False))
        add("bn", nn.BatchNorm2d(x))
        add("relu", nn.ReLU(True))
        in_channels = x
      else:
        # maxpool
        add("pool", nn.MaxPool2d(2))
    add("avgpool", nn.AvgPool2d(2))
    self.backbone = nn.Sequential(OrderedDict(layers))
    self.classifier = nn.Linear(512, 10)

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    # backbone: [N, 3, 32, 32] => [N, 512, 2, 2]
    x = self.backbone(x)

    # avgpool: [N, 512, 2, 2] => [N, 512]
    x = x.mean([2, 3])
    x = x.view(x.shape[0], -1)

    # classifier: [N, 512] => [N, 10]
    x = self.classifier(x)
    return x

def vgg_custom(path = "model_weights/pretrained_AlexNet.pt", pretrained=True, progress=True, **kwargs):
    model = VGG(**kwargs)
#     net = torch.jit.load(path)
    net = torch.load(path, map_location=torch.device('cpu'))
    params = net.state_dict()
#     state_dict = load_state_dict_from_url(model_url, progress=progress)
#     model.load_state_dict(state_dict)
    model.load_state_dict(params)
    return model


net = vgg_custom(path = "model/8bit_kmeans_quantized.pth") #pretrained on ImageNet - 1000 classes
# net = alexnet_custom(path = "model_weights/pretrained_AlexNet.pt")

In [8]:
#Model eval on CIFAR10

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(test_data, batch_size=4, shuffle=True, num_workers=2)

classes = ('Airplane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck')

from torch.utils.data import DataLoader
from tqdm.auto import tqdm

  
# net = vgg_custom(path = "model/8bit_kmeans_quantized.pth") 
# correct = 0
# total = 0
# with torch.no_grad():
#     for i, data in enumerate(testloader, 0):    #test on 1000 images
#         print(i)
#         inputs = extra_preprocess(data[0])
# #         if i==0 :
# #             net = convert_weights(net, inputs.dtype)
#         print(net.backbone.conv0.weight.dtype)
#         outputs = net(data[0])
#         _, predicted = torch.max(outputs.data, 1)
#         total += data[1].size(0)
#         correct += (predicted == data[1]).sum().item()
#         if i==100:
#             break
# print('Accuracy of the network on the 1000 test images: %.2f %%' % (100 * correct / total))


# def extra_preprocess(x):
#     return (x*255 - 128).clamp(-128, 127).to(torch.int8)

#pretrained on ImageNet - 1000 classes

# net = vgg_custom(path = "model/8bit_kmeans_quantized.pth")


Files already downloaded and verified


In [9]:
# Format input

input_tensors = torch.zeros((2500,3,224,224))
for i in range(2500):
    input_tensors[i,:,:,:] = test_data[0][0]


len(input_tensors)

2500

In [10]:
# Edit these variables to the correct values
#The below are for CIFAR10 AlexNet

num_layers = 11
layer_type = ['conv','pool','conv','pool','conv','conv','conv','pool','fc','fc','fc'
             ]

layer_input_sz = [
    (50,224,224,3),
    (50, 55, 55, 64),
    (50, 27, 27, 64),
    (50, 27, 27, 192),
    (50, 13, 13, 192),
    (50, 13, 13, 384),
    (50, 13, 13, 256),
    (50, 13, 13, 256),
    (50, 6, 6, 256),
    (50, 4096),
    (50, 4096)
]

layer_param = [
    (11,11,3,64,4,4,2),
    (3,3,2,2),
    (5,5,64,192,1,1,2),
    (3,3,2,2),
    (3,3,192,384,1,1,1),
    (3,3,384,256,1,1,1),
    (3,3,256,256,1,1,1),
    (3,3,2,2),
    (4096,256*6*6),
    (4096,4096),
    (1000,4096)
]

In [11]:
def get_output_size(inputs_sz, layer_type, layer_param):
    
    if layer_type == 'conv':
        w = int((inputs_sz[1]+2*layer_param[6]-(layer_param[0]-1)-1)/layer_param[4]) +1
        h = int((inputs_sz[2]+2*layer_param[6]-(layer_param[1]-1)-1)/layer_param[5]) +1
        return (inputs_sz[0], w, h,layer_param[3])
    
    elif layer_type == 'pool':
        w = int((inputs_sz[1]-(layer_param[0]-1)-1)/layer_param[2]) +1
        h = int((inputs_sz[2]-(layer_param[1]-1)-1)/layer_param[3]) +1
        return (inputs_sz[0], w, h,inputs_sz[3])

    elif layer_type == 'fc':
        return (inputs_sz[0], layer_param[0])


def num_params(layer_type, layer_param):
    
    if layer_type == 'conv':
        return ((layer_param[0]*layer_param[1]*layer_param[2])+1)*layer_param[3]

    elif layer_type == 'pool':
        return 0
    
    elif layer_type == 'fc':
        return (layer_param[1]*layer_param[0])+1*layer_param[0]

# Required memory in bytes
def param_memory_size(layer_type, layer_param):    
    param_numbr = num_params(layer_type, layer_param)
    return param_numbr*4


def num_mult(input_sz, layer_type, layer_param):
    
    if layer_type == 'conv':
        (batch_size, width, height, channels) = get_output_size(input_sz, layer_type, layer_param)
        return (batch_size*(width*height*channels)*(layer_param[0]*layer_param[1]*layer_param[2]))

    elif layer_type == 'pool':
        return 0
    
    elif layer_type == 'fc':
        return (input_sz[0]*layer_param[0]*layer_param[1])


layer_output_sz = []
layer_params_mem = []
layer_mult_count = []

for n in range(num_layers):
    layer_output_sz.append(get_output_size(layer_input_sz[n], layer_type[n], layer_param[n]))
    layer_params_mem.append(param_memory_size(layer_type[n], layer_param[n]))
    layer_mult_count.append(num_mult(layer_input_sz[n], layer_type[n], layer_param[n]))

print("Network Summary:")
print("# Type\tInput Size\t\tWeight Param\t\t\t\tOutput Size\t\tWeight Memory\t#mult")
for layer_idx in range(num_layers):
    print("%d %s\t%s\t%s\t%s\t%s\t%s" % (
            (layer_idx+1),
            layer_type[layer_idx], 
            str(layer_input_sz[layer_idx]).ljust(16), 
            str(layer_param[layer_idx]).ljust(32), 
            str(layer_output_sz[layer_idx]).ljust(16), 
            str(layer_params_mem[layer_idx]).ljust(12),
            str(layer_mult_count[layer_idx]).ljust(12)
        ))

Network Summary:
# Type	Input Size		Weight Param				Output Size		Weight Memory	#mult
1 conv	(50, 224, 224, 3)	(11, 11, 3, 64, 4, 4, 2)        	(50, 55, 55, 64)	93184       	3513840000  
2 pool	(50, 55, 55, 64)	(3, 3, 2, 2)                    	(50, 27, 27, 64)	0           	0           
3 conv	(50, 27, 27, 64)	(5, 5, 64, 192, 1, 1, 2)        	(50, 27, 27, 192)	1229568     	11197440000 
4 pool	(50, 27, 27, 192)	(3, 3, 2, 2)                    	(50, 13, 13, 192)	0           	0           
5 conv	(50, 13, 13, 192)	(3, 3, 192, 384, 1, 1, 1)       	(50, 13, 13, 384)	2655744     	5607014400  
6 conv	(50, 13, 13, 384)	(3, 3, 384, 256, 1, 1, 1)       	(50, 13, 13, 256)	3539968     	7476019200  
7 conv	(50, 13, 13, 256)	(3, 3, 256, 256, 1, 1, 1)       	(50, 13, 13, 256)	2360320     	4984012800  
8 pool	(50, 13, 13, 256)	(3, 3, 2, 2)                    	(50, 6, 6, 256) 	0           	0           
9 fc	(50, 6, 6, 256) 	(4096, 9216)                    	(50, 4096)      	151011328   	1887436800  
10 fc	

In [12]:
def get_activations(net, input_tensors):
    inp = input_tensors; k = "features"; act = []
    l = [0, 3, 6, 8, 10]
    for i in range(12):
        if i in l:
            act += [inp.detach()]
        inp = net.features[i](inp)
    inp = net.avgpool(inp)
    inp = inp.view(inp.size(0), 256 * 6 * 6)
    l = [1, 4, 6]
    for i in range(7):
        if i in l:
            act += [inp.detach()]
        inp = net.classifier[i](inp)
    return act

act = get_activations(net, input_tensors[1].unsqueeze(0))

AttributeError: 'VGG' object has no attribute 'features'

In [8]:
print("Layer\tTotal\tZeros\tZero Percentage")
nonzero_sum = 0
tot_sum = 0
zero_count = []
for i in range(len(act)):
    act_i = act[i].numpy()
    tot, zeros = act_i.size, np.count_nonzero(act_i==0)
    zero_count += [zeros]
    nonzero_sum += tot-zeros
    tot_sum += tot
    frac = 100.0*zeros/tot
    print("%d\t%d\t%d\t%g" % (i+1, tot, zeros, frac))

Layer	Total	Zeros	Zero Percentage
1	150528	0	0
2	46656	19027	40.7815
3	32448	14843	45.744
4	64896	53806	82.9111
5	43264	34947	80.7762
6	9216	7597	82.4327
7	4096	3365	82.1533
8	4096	3556	86.8164


In [9]:

total_mult = 0
zero_mult = 0
i=0
for n in range(num_layers):
    num_multi = num_mult(layer_input_sz[n], layer_type[n], layer_param[n])
    total_mult += num_multi
    if layer_type[n] == 'conv':
        zer = layer_input_sz[n][0]*zero_count[i]*layer_param[n][0]*layer_param[n][1] #Assuming each zero in the input activation will involve in kernel_width x kernel_height multiplications
        zero_mult += zer
        i += 1
#         print (zer)
    if layer_type[n] == 'fc':
        zer = layer_input_sz[n][0]*zero_count[i]*layer_param[n][0]
        zero_mult += zer
        i += 1
#         print (zer)
    
        
non_zero_mult = total_mult - zero_mult

frac = 100.0*non_zero_mult/total_mult
# ----------

print("Total Mul\tNon-Zero Mul\tNon-Zero Percentage")
print("%d\t%d\t%g" % (total_mult, non_zero_mult, frac))

Total Mul	Non-Zero Mul	Non-Zero Percentage
35709424000	33216204450	93.018


In [3]:
!accelergyTables

In [ ]:
# some basic imports and setups

import profiler
from profiler import Profiler
import torchvision

profiler = Profiler(
        top_dir='./../workloads',
        sub_dir='vgg_8bit',
        timeloop_dir='./../example_designs',
        design = 'simple_output_stationary',
        model=net,
        input_size=(3, 224, 224),
        batch_size=1,
        convert_fc=True,
        exception_module_names=[]
    )
results = profiler.profile()

total_energy = 0
total_cycle = 0

for layer_id, info in results.items():
    print(f"ID: {layer_id} \t Energy: {info['energy']} \t Cycle: {info['cycle']} \t Number of same architecture layers: {info['num']}")
    total_energy += info['energy'] * info['num']
    total_cycle += info['cycle'] * info['num']
    
print(f'\nTotal Energy: {total_energy} uj \nTotal Cycles: {total_cycle}')


unknown module type <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
unknown module type <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
unknown module type <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
unknown module type <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
unknown module type <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
unknown module type <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
unknown module type <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
unknown module type <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
unknown module type <class 'torch.nn.modules.pooling.AvgPool2d'>
unknown module type <class '__main__.VGG'>


running timeloop to get energy and latency...


 12%|█▎        | 1/8 [00:39<04:34, 39.18s/it]

/home/workspace/final-project/scripts/../example_designs/simple_output_stationary/vgg_8bit/layer1


 25%|██▌       | 2/8 [02:49<09:16, 92.80s/it]

/home/workspace/final-project/scripts/../example_designs/simple_output_stationary/vgg_8bit/layer2


 38%|███▊      | 3/8 [06:23<12:19, 147.96s/it]

/home/workspace/final-project/scripts/../example_designs/simple_output_stationary/vgg_8bit/layer3


 50%|█████     | 4/8 [10:23<12:17, 184.36s/it]

/home/workspace/final-project/scripts/../example_designs/simple_output_stationary/vgg_8bit/layer4
